# CIS 194: Homework 4

## Exercise 1: Wholemeal programming

Reimplement each of the following functions in a more idiomatic
Haskell style. Use wholemeal programming practices, breaking each
function into a pipeline of incremental transformations to an entire
data structure. Name your functions fun1’ and fun2’ respectively.

In [1]:
fun1 :: [Integer] -> Integer
fun1 [] = 1
fun1 (x : xs)
  | even x = (x - 2) * fun1 xs
  | otherwise = fun1 xs

In [2]:
fun2 :: Integer -> Integer
fun2 1 = 0
fun2 n
  | even n = n + fun2 (n `div` 2)
  | otherwise = fun2 (3 * n + 1)

Hint: For this problem you may wish to use the functions iterate
and takeWhile. Look them up in the Prelude documentation to see
what they do.

In [3]:
fun1' :: [Integer] -> Integer
fun1' = product . map (subtract 2) . filter even

In [4]:
assert f g x = f x == g x
assertFun1 = assert fun1 fun1'

In [5]:
assertFun1 [1..10]

True

In [6]:
assertFun1 [42]

True

In [7]:
assertFun1 []

True

In [8]:
fun2' :: Integer -> Integer
fun2' = sum . filter even . takeWhile (/=1) . iterate f
  where
    f x
      | even x = x `div` 2
      | otherwise = 3 * x + 1

In [9]:
assertFun2 = assert fun2 fun2'
all assertFun2  [1..100]

True

## Exercise 2: Folding with trees

Recall the definition of a binary tree data structure. The height of a binary tree is the length of a path from the root to the deepest node. For example, the height of a tree with a single node is 0; the
height of a tree with three nodes, whose root has two children, is 1;
and so on. A binary tree is balanced if the height of its left and right
subtrees differ by no more than 1, and its left and right subtrees are
also balanced.

You should use the following data structure to represent binary
trees. Note that each node stores an extra Integer representing the
height at that node.

In [10]:
import Data.Tree (drawTree)
import Data.Tree qualified

data Tree a
  = Leaf
  | Node Integer (Tree a) a (Tree a)
  deriving (Show, Eq)

toDataTree :: (Show a) => Tree a -> Data.Tree.Tree String
toDataTree Leaf = Data.Tree.Node "L" []
toDataTree (Node i left val right) =
  Data.Tree.Node
    (show val ++ " [i=" ++ show i ++ "]")
    [toDataTree left, toDataTree right]

printTree :: (Show a) => Tree a -> IO ()
printTree = putStrLn . drawTree . toDataTree

For this exercise, write a function

In [11]:
foldTree :: [a] -> Tree a
foldTree = foldr insert Leaf
  where
    insert x Leaf = Node 0 Leaf x Leaf
    insert x (Node h l v r)
      | height l > height r = balance $ Node h l v (insert x r)
      | otherwise = balance $ Node h (insert x l) v r

    balance (Node _ l v r) = Node h l v r
      where
        h = (+ 1) . maximum . map height $ [l, r]

    height Leaf = -1
    height (Node h _ _ _) = h

In [12]:
printTree $ foldTree [1..7]

7 [i=3]
|
+- 6 [i=2]
|  |
|  +- 4 [i=1]
|  |  |
|  |  +- 1 [i=0]
|  |  |  |
|  |  |  +- L
|  |  |  |
|  |  |  `- L
|  |  |
|  |  `- L
|  |
|  `- 2 [i=0]
|     |
|     +- L
|     |
|     `- L
|
`- 5 [i=1]
   |
   +- 3 [i=0]
   |  |
   |  +- L
   |  |
   |  `- L
   |
   `- L

In [13]:
printTree $ foldTree "ABCDEFGHIJ"

'J' [i=3]
|
+- 'I' [i=2]
|  |
|  +- 'G' [i=1]
|  |  |
|  |  +- 'D' [i=0]
|  |  |  |
|  |  |  +- L
|  |  |  |
|  |  |  `- L
|  |  |
|  |  `- L
|  |
|  `- 'E' [i=1]
|     |
|     +- 'A' [i=0]
|     |  |
|     |  +- L
|     |  |
|     |  `- L
|     |
|     `- L
|
`- 'H' [i=2]
   |
   +- 'F' [i=1]
   |  |
   |  +- 'B' [i=0]
   |  |  |
   |  |  +- L
   |  |  |
   |  |  `- L
   |  |
   |  `- L
   |
   `- 'C' [i=0]
      |
      +- L
      |
      `- L

## Exercise 3: More folds!

### 1.Implement a function

```haskell
xor :: [Bool] -> Bool
```

which returns True if and only if there are an odd number of True
values contained in the input list. It does not matter how many
False values the input list contains.

In [14]:
xor' = odd . length . filter id

In [21]:
xor' []

False

In [15]:
xor' [False, True, False]

True

In [16]:
xor' [False, True, False, False, True]

False

**Your solution must be implemented using a fold.**

In [17]:
xor :: [Bool] -> Bool
xor = foldr (/=) False

In [20]:
xor []

False

In [18]:
xor [False, True, False]

True

In [19]:
xor [False, True, False, False, True]

False

### 2. Implement map as a fold.

That is, complete the definition

```haskell
map' :: (a -> b) -> [a] -> [b]
map' f = foldr ...
```

in such a way that map’ behaves identically to the standard map
function.

In [34]:
map' :: (a -> b) -> [a] -> [b]
map' f = foldr ((:) . f) []

In [35]:
assertMap f x =  map f x ==  map' f x

In [36]:
assertMap (*3) [1..5]

True

In [37]:
assertMap (+42) []

True

### 3. (Optional) Implement foldl using foldr. 
That is, complete the definition

```haskell
myFoldl :: (a -> b -> a) -> a -> [b] -> a
myFoldl f base xs = foldr ...
```

In [42]:
-- myFoldl (-) 0 [1,2,3]
-- (\x1 g a -> g (f a x1)) 1
--   ((\x2 g a -> g (f a x2)) 2
--     ((\x3 g a -> g (f a x3)) 3 id))
myFoldl :: (a -> b -> a) -> a -> [b] -> a
myFoldl f base xs = foldr (\x g acc -> g (f acc x)) id xs base

## Exercise 4: Finding primes

Read about the Sieve of Sundaram. Implement the algorithm us ing function composition. Given an integer n, your function should generate all the odd prime numbers up to 2n + 2.

```haskell
sieveSundaram :: Integer -> [Integer] 
```

To give you some help, below is a function to compute the Cartesian product of two lists. This is similar to zip, but it produces all
possible pairs instead of matching up the list elements.

For example:

```haskell
cartProd [1,2] [’a’,’b’] == [(1,’a’),(1,’b’),(2,’a’),(2,’b’)]
```

It’s written using a list comprehension, which we haven’t talked about
in class (but feel free to research them)

In [1]:
cartProd :: [a] -> [b] -> [(a, b)]
cartProd xs ys = [(x,y) | x <- xs, y <- ys]

In [16]:
import Data.List

sieveSundaram :: Integer -> [Integer]
sieveSundaram n = 2 : primes
  where
    primes = [2 * x + 1 | x <- ls \\ sieve]
    sieve = sort . filter (<= n) $ [x + y + 2 * x * y | (x, y) <- cartProd ls ls, x <= y]
    ls = [1 .. n]

In [17]:
sieveSundaram 50

[2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101]